In [1]:
import numpy as np
import tkinter as tk
import time as time
import random as rand
import math

In [2]:
# Global
UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
MOVES = [UP, DOWN, LEFT, RIGHT]

EMPTY = 0
FOOD = 99

In [7]:
class Game:
    def __init__(self, size, num_snakes, players, gui=None, display=False, max_turns=100):
        self.size = size
        self.num_snakes = num_snakes
        self.players = players
        self.gui = gui
        self.display = display
        self.max_turns = max_turns
        
        self.num_food = 4
        self.turn = 0
        self.snake_size = 3
        
        self.snakes = [[((j + 1) * self.size // (2 * self.num_snakes), self.size // 2 + i) for i in range(self.snake_size)]
                       for j in range(self.num_snakes)]
        
        self.food = [
            (self.size // 4, self.size // 4), 
            (3 * self.size // 4, self.size // 4), 
            (self.size // 4, 3 * self.size // 4), 
            (3 * self.size // 4, 3 * self.size // 4)
        ]
        
        self.player_ids = [i for i in range(self.num_snakes)]
        
        self.board = np.zeros([self.size, self.size])
        
        for pid in self.player_ids:
            for pos in self.snakes[pid]:
                self.board[pos[0], pos[1]] = pid + 1
        
        for pos in self.food:
            self.board[pos[0], pos[1]] = FOOD
            
        self.food_index = 0
        self.food_pos = [(4, 7), (9, 4), (9, 7), (3, 5), (0, 4), (5, 9), (2, 0), (9, 3), (4, 5), (9, 6), (0, 3), (3, 6), (2, 8), (7, 4), (5, 8), (2, 0), (0, 1), (1, 5), (0, 7), (9, 9), (8, 3), (5, 8), (2, 1), (7, 6), (1, 2), (6, 7), (8, 6), (6, 1), (6, 2), (1, 5), (6, 3), (6, 2), (2, 7), (9, 9), (9, 3), (4, 5), (5, 9), (1, 0), (9, 3), (6, 9), (9, 1), (0, 6), (1, 1), (2, 1), (1, 2), (8, 7), (4, 4), (7, 3), (1, 5), (5, 3), (5, 8), (3, 9), (4, 6), (2, 4), (3, 0), (7, 1), (9, 8), (9, 5), (9, 3), (7, 2), (7, 9), (0, 5), (2, 7), (8, 6), (6, 8), (8, 4), (6, 6), (5, 8), (3, 5), (9, 0), (4, 4), (5, 0), (8, 9), (3, 5), (9, 5), (0, 7), (3, 0), (2, 7), (2, 0), (0, 3), (9, 0), (5, 6), (2, 1), (3, 5), (9, 4), (0, 2), (2, 0), (1, 9), (8, 3), (3, 4), (9, 3), (7, 8), (4, 9), (1, 1), (2, 1), (5, 3), (1, 0), (6, 4), (9, 4), (2, 7), (0, 8), (1, 2), (1, 8), (9, 9), (4, 2), (3, 3), (1, 9), (0, 4), (0, 4), (8, 2), (7, 6), (0, 8), (6, 9), (7, 2), (9, 4), (9, 9), (3, 8), (4, 2), (1, 0), (5, 2), (1, 9), (3, 0), (6, 6), (8, 0), (1, 8), (6, 1), (2, 5), (5, 2), (1, 4), (1, 8), (2, 7), (8, 4), (9, 6), (1, 0), (0, 8), (9, 6), (4, 1), (6, 4), (8, 0), (6, 7), (3, 3), (2, 8), (7, 2), (2, 8), (0, 6), (6, 0), (4, 8), (6, 9), (5, 6), (0, 6), (1, 8), (0, 5), (2, 3), (1, 6), (1, 4), (1, 1), (8, 5), (2, 0), (7, 2), (6, 4), (4, 2), (4, 9), (0, 0), (5, 8), (8, 9), (6, 5), (2, 9), (6, 6), (0, 9), (3, 7), (9, 5), (0, 9), (4, 7), (8, 6), (2, 8), (7, 2), (6, 9), (1, 1), (8, 2), (3, 5), (1, 8), (4, 8), (8, 6), (2, 4), (7, 1), (1, 9), (6, 7), (3, 3), (2, 9), (8, 8), (6, 9), (2, 2), (6, 8), (2, 7), (5, 5), (3, 9), (3, 2), (4, 4), (0, 8), (1, 2)]
    
    def move(self):
        moves = []
        # Moves the head
        for i in self.player_ids:
            snake_i = self.snakes[i]
            move_i = self.players[i].get_move(self.board, snake_i)
            moves.append(move_i)
            new_square = (snake_i[-1][0] + move_i[0], snake_i[-1][1] + move_i[1])
            snake_i.append(new_square)
            
        # update tail
        for pid in self.player_ids:
            head_i = self.snakes[pid][-1]
            if head_i not in self.food:
                self.board[self.snakes[i][0][0]][self.snakes[i][0][1]] = EMPTY
                self.snakes[i].pop(0)
            else:
#                 print('Remove food: ({}, {})'.format(head_i[0], head_i[1]))
                self.food.remove(head_i)
                
        # check oob
        for i in self.player_ids:
            head_i = self.snakes[i][-1]
            if head_i[0] >= self.size or head_i[1] >= self.size or head_i[0] < 0 or head_i[1] < 0:
                self.player_ids.remove(i)
            else:
                self.board[head_i[0]][head_i[1]] = i + 1
                
        # check 
        for i in self.player_ids:
            head_i = self.snakes[i][-1]
            for j in range(self.num_snakes):
                if i == j and head_i in self.snakes[i][:-1] or i != j and head_i in self.snakes[j]:
                    self.player_ids.remove(i)
                    
        # spawn new food
        while len(self.food) < self.num_food:
            x = self.food_pos[self.food_index][0]
            y = self.food_pos[self.food_index][1]
            while self.board[x][y] != EMPTY:
                self.food_index += 1
                x = self.food_pos[self.food_index][0]
                y = self.food_pos[self.food_index][1]
            
#             print('Add food: ({}, {})'.format(x, y))
            self.food.append((x, y))
            self.board[x][y] = FOOD
            self.food_index += 1
        
        return moves
        
    def play(self, display, termination=False):
        if display:
            self.display_board()
        
        while True:
            if termination:
                if display:
                    self.gui.app.destroy()
                for i in self.player_ids:
                    if len(self.snakes[0]) - self.turn / 20 <= 0:
                        self.player_ids.remove(i)
                        return -2
            if len(self.player_ids) == 0:
                return -1
            if self.turn >= self.max_turns:
                return 0
            moves = self.move()
            self.turn += 1
            if display:
                for move in moves:
                    if move == UP:
                        print('UP')
                    elif move == RIGHT:
                        print('RIGHT')
                    elif move == LEFT:
                        print('LEFT')
                    else:
                        print('DOWN')
                self.display_board()
                if self.gui is not None:
                    self.gui.update()
                
                time.sleep(1)
                    
    def display_board(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == EMPTY:
                    print('|_', end='')
                elif self.board[i][j] == FOOD:
                    print('|#', end='')
                else:
                    print('|{}'.format(int(self.board[i][j])), end='')
            print('|')

In [4]:
class RandomPlayer:
    def __init__(self, i):
        self.i = i
        
    def get_move(self, board, snake):
        r = rand.randint(0, 3)
        return MOVES[r]
    
class GeneticPlayer:
    def __init__(self, pop_size, num_generations, num_trials, window_size, hidden_size, board_size, mut_rate=0.1, mut_size=0.1):
        self.pop_size = pop_size
        self.num_generations = num_generations
        self.num_trials = num_trials
        self.window_size = window_size # surrounding area of snake head
        self.hidden_size = hidden_size
        self.board_size = board_size
        self.mut_rate = mut_rate
        self.mut_size = mut_size
        
        self.display = False # for debugging
        
        self.current_brain = None # brain = neural network
        input_size = self.window_size**2
        output_size = len(MOVES)
        self.pop = [self.generate_brain(input_size, hidden_size, output_size) for _ in range(self.pop_size)]
        
    def generate_brain(self, input_size, hidden_size, output_size):
        hidden_layer1 = np.array([[rand.uniform(-1, 1) for _ in range(input_size + 1)] for _ in range(hidden_size)])
        hidden_layer2 = np.array([[rand.uniform(-1, 1) for _ in range(hidden_size + 1)] for _ in range(hidden_size)])
        output_layer = np.array([[rand.uniform(-1, 1) for _ in range(hidden_size + 1)] for _ in range(output_size)])
        return [hidden_layer1, hidden_layer2, output_layer]
    
    def get_move(self, board, snake):
        input_vector = self.process_board(board, snake[-1][0], snake[-1][1], snake[-2][0], snake[-2][1]) # assumes snake len > 1; then get coors of box next to head for coors
        hidden_layer1 = self.current_brain[0]
        hidden_layer2 = self.current_brain[1]
        output_layer = self.current_brain[2]
        
        # feedforeward pass
        # tanh to normalize between -1 to 1
        # [1] is bias
        hidden_result1 = np.array([math.tanh(np.dot(input_vector, hidden_layer1[i])) for i in range(hidden_layer1.shape[0])] + [1])
        hidden_result2 = np.array([math.tanh(np.dot(hidden_result1, hidden_layer2[i])) for i in range(hidden_layer2.shape[0])] + [1])
        output_result = np.array([math.tanh(np.dot(hidden_result2, output_layer[i])) for i in range(output_layer.shape[0])])
        
        max_index = np.argmax(output_result)
        return MOVES[max_index]
    
    def process_board(self, board, x1, y1, x2, y2):
        input_vector = [[0 for _ in range(self.window_size)] for _ in range(self.window_size)]
        
        for offset_x in range(self.window_size):
            for offset_y in range(self.window_size):
                cur_x = x1 + offset_x - self.window_size // 2
                cur_y = y1 + offset_y - self.window_size // 2
                if cur_x < 0 or cur_y < 0 or cur_x >= self.board_size or cur_y >= self.board_size:
                    input_vector[offset_x][offset_y] = -1
                elif board[cur_x][cur_y] == FOOD: # this should be [cur_y][cur_x] since y== row, x == col right?
                    input_vector[offset_x][offset_y] = 1
                elif board[cur_x][cur_y] == EMPTY:
                    input_vector[offset_x][offset_y] = 0
                else: # it is another snake
                    input_vector[offset_x][offset_y] = -1
        
        input_vector = list(np.array(input_vector).flatten()) + [1]
        
        if self.display:
            print(np.array(input_vector))
        
        return np.array(input_vector)

    def selection(self, candids):
        new_pop = [] # 1/4 top candids, 1/4 mutations, 1/4 random
        for brain in candids:
            new_pop.append(brain)
            new_pop.append(self.mutate(brain))
        
        # spawn random brains
        for _ in range(self.pop_size // 2):
            new_pop.append(self.generate_brain(self.window_size**2, self.hidden_size, len(MOVES)))

#         for i in range(self.pop_size):
#             ind = int(rand.uniform(0, len(candids)))
#             new_pop.append(self.mutate(candids[ind]))
        
        return new_pop
    
    def mutate(self, brain):
        new_brain = []
        for layer in brain:
            new_layer = np.copy(layer)
            for i in range(new_layer.shape[0]):
                for j in range(new_layer.shape[1]):
                    if rand.uniform(0, 1) < self.mut_rate:
                        new_layer[i][j] += rand.uniform(-1, 1) # * self.mut_size
            new_brain.append(new_layer)
        return new_brain
    
    def run(self):
        scores = [0 for _ in range(self.pop_size)]
        
        #DEBUG
        max_score = 0
        for i in range(self.pop_size):
            for j in range(self.num_trials):
                self.current_brain = self.pop[i]
                game = Game(self.board_size, 1, [self])
                outcome = game.play(False, termination=True)
                score = len(game.snakes[0]) # for single player 
                scores[i] += score # add scores over all trials
                
                ## DEBUG
                if outcome == 0:
                    print('Snake {} made it to the last turn'.format(i))
                    
                if score > max_score:
                    max_score = score
#                     print('{} at ID {}'.format(score, i))
            
        print('Max score: {}'.format(max_score))
        candid_indices = list(np.argsort(scores))[-self.pop_size // 4:] # get top 25%
#         print('Top scores: {}'.format(list(np.sort(scores))))
        candids = [self.pop[i] for i in candid_indices][::-1] # reverse list so top brain is first
#         print('Top brain: {}'.format(candids[0]))
        self.pop = self.selection(candids)
        
    def evolve(self):
        for i in range(self.num_generations):
            self.run()
            print('Generation: {}'.format(i))
            
        # DEBUG; display boards for top brains
        key = input('Enter any character to display boards (q to quit)')
        if key == 'q':
            return
        for brain in self.pop:
            self.display = True
            self.current_brain = brain
            gui = Gui(game, 100)
            game = Game(self.board_size, 1, [self], display=True, gui=gui)
            game.play(True, termination=True)
            print('Snake length: {}'.format(len(game.snakes[0])))

In [5]:
class Gui:
    def __init__(self, game, size):
        self.game = game
        self.game.gui = self
        self.size = size
        
        self.ratio = self.size / self.game.size
        
        self.app = tk.Tk()
        self.canvas = tk.Canvas(self.app, width=self.size, height=self.size)
        self.canvas.pack()
        

        # draw snakes and food
        self.draw_snakes_and_food()
    
    def draw_snakes_and_food(self):
        for i, snake in enumerate(self.game.snakes):
            color = '#' + '{0:03X}'.format((i + 1) * 500)
            self.canvas.create_rectangle(self.ratio * snake[-1][1], self.ratio * snake[-1][0], 
                                         self.ratio * (snake[-1][1] + 1), self.ratio * (snake[-1][0] + 1), fill=color)
            
            # draw rest of body
            for j in range(len(snake) - 1):
                color = '#' + '{0:03X}'.format((i + 1) * 123)
                self.canvas.create_rectangle(self.ratio * snake[j][1], self.ratio * snake[j][0], 
                             self.ratio * (snake[j][1] + 1), self.ratio * (snake[j][0] + 1), fill=color)
                
        for food in self.game.food:
            self.canvas.create_rectangle(self.ratio * food[1], self.ratio * food[0], 
                                         self.ratio * (food[1] + 1), self.ratio * (food[0] + 1), fill = '#000')
    
    def update(self):
        self.canvas.delete('all')
        self.draw_snakes_and_food()
        self.canvas.pack()
        self.app.update()

In [ ]:
# size = 10
# num_snakes = 1

# players = [RandomPlayer(0)]

# game = Game(size, num_snakes, players, gui=None, display=True, max_turns=100)

# gui_size = 800
# gui = Gui(game, gui_size)
# game.play(True, termination=False)


pop_size = 100
num_generations = 1500
num_trials = 1
window_size = 7
hidden_size = 15
board_size = 10
gen_player = GeneticPlayer(pop_size, num_generations, num_trials, window_size, hidden_size, board_size, mut_rate=0.1, mut_size=0.1)
gen_player.evolve()

Max score: 5
Generation: 0
Max score: 5
Generation: 1
Max score: 5
Generation: 2
Max score: 5
Generation: 3
Max score: 5
Generation: 4
Max score: 5
Generation: 5
Max score: 5
Generation: 6
Max score: 5
Generation: 7
Max score: 5
Generation: 8
Max score: 5
Generation: 9
Max score: 5
Generation: 10
Max score: 5
Generation: 11
Max score: 5
Generation: 12
Max score: 5
Generation: 13
Max score: 5
Generation: 14
Max score: 5
Generation: 15
Max score: 5
Generation: 16
Max score: 5
Generation: 17
Max score: 5
Generation: 18
Max score: 5
Generation: 19
Max score: 5
Generation: 20
Max score: 5
Generation: 21
Max score: 5
Generation: 22
Max score: 5
Generation: 23
Max score: 5
Generation: 24
Max score: 5
Generation: 25
Max score: 5
Generation: 26
Max score: 5
Generation: 27
Max score: 5
Generation: 28
Max score: 5
Generation: 29
Max score: 5
Generation: 30
Max score: 5
Generation: 31
Max score: 5
Generation: 32
Max score: 5
Generation: 33
Max score: 5
Generation: 34
Max score: 5
Generation: 35
Ma

Max score: 9
Generation: 268
Max score: 9
Generation: 269
Max score: 9
Generation: 270
Max score: 9
Generation: 271
Max score: 9
Generation: 272
Max score: 9
Generation: 273
Max score: 9
Generation: 274
Max score: 9
Generation: 275
Snake 45 made it to the last turn
Max score: 9
Generation: 276
Max score: 9
Generation: 277
Max score: 9
Generation: 278
Max score: 9
Generation: 279
Max score: 9
Generation: 280
Max score: 9
Generation: 281
Max score: 9
Generation: 282
Max score: 9
Generation: 283
Max score: 9
Generation: 284
Max score: 9
Generation: 285
Max score: 9
Generation: 286
Max score: 9
Generation: 287
Max score: 9
Generation: 288
Max score: 9
Generation: 289
Max score: 9
Generation: 290
Max score: 9
Generation: 291
Max score: 9
Generation: 292
Max score: 9
Generation: 293
Max score: 9
Generation: 294
Max score: 9
Generation: 295
Max score: 9
Generation: 296
Max score: 9
Generation: 297
Max score: 9
Generation: 298
Max score: 9
Generation: 299
Max score: 9
Generation: 300
Max score

Snake 48 made it to the last turn
Max score: 13
Generation: 471
Snake 5 made it to the last turn
Snake 30 made it to the last turn
Max score: 13
Generation: 472
Snake 18 made it to the last turn
Max score: 13
Generation: 473
Snake 44 made it to the last turn
Max score: 13
Generation: 474
Snake 9 made it to the last turn
Snake 14 made it to the last turn
Max score: 13
Generation: 475
Max score: 13
Generation: 476
Snake 21 made it to the last turn
Max score: 13
Generation: 477
Snake 9 made it to the last turn
Max score: 13
Generation: 478
Max score: 13
Generation: 479
Snake 11 made it to the last turn
Max score: 13
Generation: 480
Max score: 13
Generation: 481
Snake 13 made it to the last turn
Max score: 13
Generation: 482
Snake 27 made it to the last turn
Max score: 13
Generation: 483
Snake 41 made it to the last turn
Max score: 13
Generation: 484
Snake 37 made it to the last turn
Max score: 13
Generation: 485
Max score: 13
Generation: 486
Snake 13 made it to the last turn
Max score: 13

Max score: 13
Generation: 566
Snake 16 made it to the last turn
Snake 39 made it to the last turn
Max score: 13
Generation: 567
Snake 10 made it to the last turn
Max score: 13
Generation: 568
Snake 18 made it to the last turn
Max score: 13
Generation: 569
Snake 13 made it to the last turn
Snake 14 made it to the last turn
Max score: 13
Generation: 570
Snake 16 made it to the last turn
Max score: 13
Generation: 571
Snake 20 made it to the last turn
Max score: 13
Generation: 572
Snake 12 made it to the last turn
Snake 17 made it to the last turn
Max score: 13
Generation: 573
Snake 20 made it to the last turn
Max score: 13
Generation: 574
Snake 14 made it to the last turn
Snake 25 made it to the last turn
Max score: 13
Generation: 575
Snake 22 made it to the last turn
Max score: 13
Generation: 576
Snake 12 made it to the last turn
Snake 13 made it to the last turn
Max score: 13
Generation: 577
Snake 19 made it to the last turn
Snake 20 made it to the last turn
Max score: 13
Generation: 57

Snake 42 made it to the last turn
Snake 44 made it to the last turn
Max score: 14
Generation: 634
Snake 9 made it to the last turn
Snake 24 made it to the last turn
Snake 43 made it to the last turn
Snake 46 made it to the last turn
Max score: 14
Generation: 635
Snake 46 made it to the last turn
Max score: 14
Generation: 636
Snake 48 made it to the last turn
Max score: 14
Generation: 637
Snake 40 made it to the last turn
Max score: 14
Generation: 638
Snake 32 made it to the last turn
Max score: 14
Generation: 639
Snake 38 made it to the last turn
Snake 39 made it to the last turn
Max score: 14
Generation: 640
Snake 38 made it to the last turn
Snake 47 made it to the last turn
Max score: 14
Generation: 641
Snake 36 made it to the last turn
Snake 47 made it to the last turn
Max score: 14
Generation: 642
Snake 36 made it to the last turn
Snake 41 made it to the last turn
Max score: 15
Generation: 643
Max score: 15
Generation: 644
Max score: 15
Generation: 645
Max score: 15
Generation: 646

...

Snake 36 made it to the last turn
Snake 39 made it to the last turn
Snake 48 made it to the last turn
Max score: 19
Generation: 1429
Snake 2 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 14 made it to the last turn
Snake 16 made it to the last turn
Snake 24 made it to the last turn
Snake 26 made it to the last turn
Snake 28 made it to the last turn
Snake 32 made it to the last turn
Snake 34 made it to the last turn
Snake 36 made it to the last turn
Snake 37 made it to the last turn
Snake 38 made it to the last turn
Snake 39 made it to the last turn
Max score: 19
Generation: 1430
Snake 2 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 8 made it to the last turn
Snake 10 made it to the last turn
Snake 11 made it to the last turn
Snake 12 made it to the last turn
Snake 16 made it to the last turn
Snake 18 made it to the last turn
Snake 23 made it to the last turn
Snake 26 made it to the las

Snake 47 made it to the last turn
Max score: 19
Generation: 1446
Snake 2 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 12 made it to the last turn
Snake 16 made it to the last turn
Snake 20 made it to the last turn
Snake 21 made it to the last turn
Snake 22 made it to the last turn
Snake 24 made it to the last turn
Snake 28 made it to the last turn
Snake 32 made it to the last turn
Snake 36 made it to the last turn
Snake 43 made it to the last turn
Max score: 19
Generation: 1447
Snake 1 made it to the last turn
Snake 2 made it to the last turn
Snake 3 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 8 made it to the last turn
Snake 10 made it to the last turn
Snake 14 made it to the last turn
Snake 15 made it to the last turn
Snake 18 made it to the last turn
Snake 22 made it to the last turn
Snake 23 made it to the last turn
Snake 24 made it to the last turn
Snake 32 made it to the last 

Snake 28 made it to the last turn
Snake 30 made it to the last turn
Snake 32 made it to the last turn
Snake 36 made it to the last turn
Snake 38 made it to the last turn
Snake 46 made it to the last turn
Snake 47 made it to the last turn
Max score: 19
Generation: 1462
Snake 1 made it to the last turn
Snake 2 made it to the last turn
Snake 3 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 7 made it to the last turn
Snake 9 made it to the last turn
Snake 10 made it to the last turn
Snake 12 made it to the last turn
Snake 16 made it to the last turn
Snake 18 made it to the last turn
Snake 20 made it to the last turn
Snake 32 made it to the last turn
Snake 34 made it to the last turn
Snake 36 made it to the last turn
Snake 45 made it to the last turn
Snake 48 made it to the last turn
Snake 49 made it to the last turn
Max score: 19
Generation: 1463
Snake 1 made it to the last turn
Snake 2 made it to the last turn
Snake 4 made it to the last t

Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 13 made it to the last turn
Snake 14 made it to the last turn
Snake 18 made it to the last turn
Snake 20 made it to the last turn
Snake 22 made it to the last turn
Snake 24 made it to the last turn
Snake 30 made it to the last turn
Snake 32 made it to the last turn
Snake 34 made it to the last turn
Snake 40 made it to the last turn
Snake 47 made it to the last turn
Snake 48 made it to the last turn
Max score: 19
Generation: 1478
Snake 2 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 8 made it to the last turn
Snake 12 made it to the last turn
Snake 18 made it to the last turn
Snake 20 made it to the last turn
Snake 22 made it to the last turn
Snake 28 made it to the last turn
Snake 30 made it to the last turn
Snake 32 made it to the last turn
Snake 42 made it to the last turn
Snake 46 made it to the last turn
Max score: 19
Generation: 1479
Snake 1 made it to the las

Snake 2 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 8 made it to the last turn
Snake 12 made it to the last turn
Snake 14 made it to the last turn
Snake 15 made it to the last turn
Snake 22 made it to the last turn
Snake 24 made it to the last turn
Snake 26 made it to the last turn
Snake 28 made it to the last turn
Snake 31 made it to the last turn
Snake 38 made it to the last turn
Snake 40 made it to the last turn
Snake 43 made it to the last turn
Snake 48 made it to the last turn
Snake 49 made it to the last turn
Max score: 19
Generation: 1493
Snake 2 made it to the last turn
Snake 4 made it to the last turn
Snake 6 made it to the last turn
Snake 12 made it to the last turn
Snake 14 made it to the last turn
Snake 24 made it to the last turn
Snake 26 made it to the last turn
Snake 28 made it to the last turn
Snake 32 made it to the last turn
Snake 36 made it to the last turn
Snake 38 made it to the last turn
Snake 42 made it to the 

In [ ]:
# game.gui.app.destroy()